# Mapping

Try a RAG approach of mapping incoming message formats to the common data format.

## Setup

In [11]:
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

  Using cached boto3-1.33.9-py3-none-any.whl.metadata (6.7 kB)
  Using cached awscli-1.31.9-py3-none-any.whl.metadata (11 kB)
  Using cached botocore-1.33.9-py3-none-any.whl.metadata (6.1 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached s3transfer-0.8.2-py3-none-any.whl.metadata (1.8 kB)
  Using cached docutils-0.16-py2.py3-none-any.whl (548 kB)
  Using cached PyYAML-6.0.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
  Using cached rsa-4.7.2-py3-none-any.whl (34 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached urllib3-2.0.7-py3-none-any.whl.metadata (6.6 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached pyasn1-0.5.1-py2.py3-none-any.whl.metadata (8.6 kB)
Using cached boto3-1.33.9-py3-none-any.whl (139 kB)
Using cached awscli-1.31.9-py3-none-any.whl (4.3 MB)
Using cached botocore-1.33.9-py3-none-any.whl (11.8 MB)
Using cached PyYAML-6.0.

In [17]:
%pip install  \
    langchain==0.0.309 \
    "transformers>=4.24,<5" \
    sqlalchemy -U \
    "faiss-cpu>=1.7,<2" \
    apache-beam \
    datasets \
    tiktoken \
    "ipywidgets>=7,<8" \
	"unstructured[md]" 


  Using cached langchain-0.0.309-py3-none-any.whl.metadata (15 kB)
  Using cached transformers-4.35.2-py3-none-any.whl.metadata (123 kB)
  Using cached SQLAlchemy-2.0.23-cp310-cp310-macosx_11_0_arm64.whl.metadata (9.6 kB)
  Using cached faiss_cpu-1.7.4-cp310-cp310-macosx_11_0_arm64.whl (2.7 MB)
  Using cached apache_beam-2.52.0-cp310-cp310-macosx_11_0_arm64.whl
  Using cached datasets-2.15.0-py3-none-any.whl.metadata (20 kB)
  Using cached tiktoken-0.5.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached ipywidgets-7.8.1-py2.py3-none-any.whl.metadata (1.9 kB)
  Using cached unstructured-0.11.2-py3-none-any.whl.metadata (25 kB)
  Using cached aiohttp-3.9.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (7.4 kB)
  Using cached anyio-3.7.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached dataclasses_json-0.6.3-py3-none-any.whl.metadata (25 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0

In [18]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    #assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


## Configure langchain

We begin with instantiating the LLM and the Embeddings model. Here we are using Anthropic Claude for text generation and Amazon Titan for text embedding.


In [20]:
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock


llm = Bedrock(model_id="anthropic.claude-v2:1", client=boto3_bedrock, model_kwargs={'max_tokens_to_sample':5000})
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=boto3_bedrock)

We need to add the embeddings of our known mappings to the Vector store. The Claude 2.1 FM has a large 200k token input limit therefore we don't need to worry about splitting the templates into smaller chunks to fit.

In [21]:
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader('./mappings', glob="**/*.md", show_progress=True)
docs = loader.load()

avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in docs])//len(docs)
avg_char_count = avg_doc_length(docs)
print(f'Average length among {len(docs)} documents loaded is {avg_char_count} characters.')

 50%|█████     | 5/10 [00:04<00:04,  1.17it/s]

Average length among 5 documents loaded is 1419 characters.


Sample the embeddings for one of the mappings.

In [23]:
import numpy as np

sample_embedding = np.array(bedrock_embeddings.embed_query(docs[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)


Sample embedding of a document chunk:  [ 0.5154619  -0.11262664  0.7485921  ...  0.10680589  0.02426382
 -0.00131729]
Size of the embedding:  (1536,)


As this is a quick prototype, use FAISS (in-memory vector store) within LangChain. But use OpenSearch Serverless for the hackathon.

In [24]:
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

def create_vector_store() :
    vectorstore_faiss = FAISS.from_documents(
        docs,
        bedrock_embeddings,
    )
    # wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss)
    return vectorstore_faiss

vector_store = create_vector_store()


## Obtain the mapped result

Helper functions:

In [25]:
import pprint

def execute(prompt, query, expected) :
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_store.as_retriever(
            search_type="similarity", search_kwargs={"k": 3}
            # search_type="similarity_score_threshold", search_kwargs={"score_threshold": .9}
        ),
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt}
    )
    answer = qa({"query": query})

    if expected != '?':
        print("Expected: \n", expected)
        print("\nActual: \n", answer['result'])
    else:
        print(answer['result'])
    
    # print("\tquery: \n", answer['query'])
    # print("\nSource_documents: \n", answer['source_documents'])


Now that we have our vector store in place, we can start asking questions. Let's define a reusable template.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
prompt_template = """

Human: Use the context within the following <context></context> XML tag to provide a concise answer to the question at the end:
<context>
{context}
</context

<schema>
{{
  "$schema": "https://json-schema.org/draft/2020-12",
  "type": "object",
  "properties": {{
    "sensor_id": {{
      "type": "string"
    }},
    "timestamp": {{
      "type": "string",
      "format": "datetime"
    }},
    "pm0_3": {{
      "type": "number"
    }},
    "pm0_5": {{
      "type": "number"
    }},
    "pm1": {{
      "type": "number"
    }},
    "pm2_5": {{
      "type": "number"
    }},
    "pm4": {{
      "type": "number"
    }},
    "pm5": {{
      "type": "number"
    }},
    "pm10": {{
      "type": "number"
    }},
    "temperature": {{
      "type": "number"
    }},
    "humidity": {{
      "type": "number"
    }}
  }},
  "required": [
    "sensor_id",
    "timestamp"
  ]
}}
</schema>

[Task instructions]
You ALWAYS follow these guidelines when writing your response:
<guidelines>
- You will be acting as an expert software developer, writing responses as json in the AFRI_SET_COMMON json format. 
- Return only the converted json as the response, along with a confidence (as a percentage) of how well you did. The json response must adhere to the json schema defined in the <schema></schema> XML tag.
- Do not return any other surrounding text, explanation or context.
</guidelines>

When you reply, first determine how the provided input should be mapped to the AFRI_SET_COMMON json format. Write this mapping within the <thinking></thinking> XML tags. This is a space for you to write down relevant content and will not be shown to the user.  Once you are done extracting determing the mapping steps, answer the question.  Put your answer inside the <response></response> XML tags.

Question: {question}

Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

: 

Define the question(s) we want to ask.

In [ ]:
# This is an example from air-gradient-dc5475b0f97c.csv:

query_1 = """Map the following provided data to the AFRI_SET_COMMON formmat:
locationId,locationName,pm01,pm02,pm10,pm003Count,atmp,rhum,rco2,tvoc,wifi,timestamp,serialno,firmwareVersion,tvocIndex,noxIndex,datapoints
59513,dc5475b0f97c,15.6,26.400002,27.3,2994.5,25.3,70.8,,,-69,2023-11-12T01:20:00.000Z,dc5475b0f97c,,,,2
59513,dc5475b0f97d,10.65,18.349998,18.75,2125.5,24.6,73.75,,,-68.5,2023-11-12T02:35:00.000Z,dc5475b0f97c,,,,2
"""

# this is just used for testing. The model never sees this
expected = [{
    'sensor_id': "dc5475b0f97c",
    'timestamp': "2023-11-12T01:20:00.000Z",
    'pm1': 15.6,
    'pm2_5': 26.400002,
    'pm10': 27.3,
    'temperature': 25.3,
    'humidity': 70.8
},{
    'sensor_id': "dc5475b0f97d",
    'timestamp': "2023-11-12T02:35:00.000Z",
    'pm1': 10.65,
    'pm2_5': 18.349998,
    'pm10': 18.75,
    'temperature': 24.6,
    'humidity': 73.75
}]

execute(PROMPT, query_1, expected)

: 

In [ ]:
# This is an example from airbeam.csv

query_2 = """Map the following provided data to the AFRI_SET_COMMON formmat:
,,,,,Sensor_Package_Name,Sensor_Package_Name,Sensor_Package_Name,Sensor_Package_Name,Sensor_Package_Name
,,,,,AirBeam3-943cc67daabc,AirBeam3-943cc67daabc,AirBeam3-943cc67daabc,AirBeam3-943cc67daabc,AirBeam3-943cc67daabc
,,,,,Sensor_Name,Sensor_Name,Sensor_Name,Sensor_Name,Sensor_Name
,,,,,AirBeam3-F,AirBeam3-PM1,AirBeam3-PM10,AirBeam3-PM2.5,AirBeam3-RH
,,,,,Measurement_Type,Measurement_Type,Measurement_Type,Measurement_Type,Measurement_Type
,,,,,Temperature,Particulate Matter,Particulate Matter,Particulate Matter,Humidity
,,,,,Measurement_Units,Measurement_Units,Measurement_Units,Measurement_Units,Measurement_Units
,,,,,fahrenheit,microgram per cubic meter,microgram per cubic meter,microgram per cubic meter,percent
ObjectID,Session_Name,Timestamp,Latitude,Longitude,1:Measurement_Value,2:Measurement_Value,3:Measurement_Value,4:Measurement_Value,5:Measurement_Value
421,AfriSET (1),2023-10-06T21:55:17.000,5.65151,-0.185649,90.0,7.0,8.0,8.5,69.0
20,AfriSET (1),2023-10-06T15:13:20.000,5.65151,-0.185649,100.0,5.0,6.0,7.0,37.0
"""

# this is just used for testing. The model never sees this
expected = [{
    'sensor_id': "AirBeam3-943cc67daabc",
    'timestamp': "2023-10-06T21:55:17.000",
    'pm1': 7.0,
    'pm2_5': 8.5,
    'pm10': 8.0,
    'temperature': 90.0,
    'humidity': 69.0
}, {
    'sensor_id': "AirBeam3-943cc67daabc",
    'timestamp': "2023-10-06T15:13:20.000",
    'pm1': 5.0,
    'pm2_5': 7.0,
    'pm10': 6.0,
    'temperature': 100.0,
    'humidity': 37.0
}]

execute(PROMPT, query_2, expected)

: 

In [ ]:
# This is an example from atmos.csv:

query_3 = """Map the following provided data to the AFRI_SET_COMMON formmat:
pm4cnc,pm4cnt,dt_time,pm25raw,pm2.5cnc,temp,rh,o3op1,o3op2,no2op1,no2op2,pm10cnc,PM10,pres,altd,pm1cnc,pm0.3cnt,pm0.5cnt,pm1cnt,pm2.5cnt,pm5cnt,pm10cnt,lat,lon,no,nox,nh3,co,co2,benzene,deviceid
0,0,2023-11-29 05:20:00,9,9,27.7,87.5,0,0,0,0,9,0,1002,308,8,0,59,69,69,0,69,0,0,0,0,0,0,0,0,3083988F1432
0,0,2023-11-29 05:21:00,8,8,27.7,87.9,0,0,0,0,8,0,1001,309,8,0,59,68,68,0,68,0,0,0,0,0,0,0,0,3083988F1432
0,0,2023-11-29 05:22:00,9,9,27.7,87.4,0,0,0,0,9,0,1001,308,8,0,64,74,74,0,75,0,0,0,0,0,0,0,0,3083988F1432
"""

# this is just used for testing. The model never sees this
expected = [{
    'sensor_id': "3083988F1432",
    'timestamp': "2023-11-29T05:20:00.000Z",
    'pm0_3': 0,
    'pm0_5': 59,
    'pm1': 69,
    'pm2_5': 69,
    'pm4': 0,
    'pm5': 0,
    'pm10': 69,
    'temperature': 27.7,
    'humidity': 87.5
}, {
    'sensor_id': "3083988F1432",
    'timestamp': "2023-11-29T05:21:00.000Z",
    'pm0_3': 0,
    'pm0_5': 59,
    'pm1': 68,
    'pm2_5': 68,
    'pm4': 0,
    'pm5': 0,
    'pm10': 68,
    'temperature': 27.7,
    'humidity': 87.9
}, {
    'sensor_id': "3083988F1432",
    'timestamp': "2023-11-29T05:22:00.000Z",
    'pm0_3': 0,
    'pm0_5': 64,
    'pm1': 74,
    'pm2_5': 74,
    'pm4': 0,
    'pm5': 0,
    'pm10': 75,
    'temperature': 27.7,
    'humidity': 87.4
}]

execute(PROMPT, query_3, expected)

: 

# Ask for how to map, instead of just the result

Create a prompt to generate something we can execute to carry out the mapping.

In [ ]:
vector_store = create_vector_store()

: 

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

mapping_prompt_template = """

Human: Use the context within the following <context></context> XML tag to provide a concise answer to the question at the end:
<context>
{context}
</context

<schema>
{{
  "$schema": "https://json-schema.org/draft/2020-12",
  "type": "object",
  "properties": {{
    "sensor_id": {{
      "type": "string"
    }},
    "timestamp": {{
      "type": "string",
      "format": "datetime"
    }},
    "pm0_3": {{
      "type": "number"
    }},
    "pm0_5": {{
      "type": "number"
    }},
    "pm1": {{
      "type": "number"
    }},
    "pm2_5": {{
      "type": "number"
    }},
    "pm4": {{
      "type": "number"
    }},
    "pm5": {{
      "type": "number"
    }},
    "pm10": {{
      "type": "number"
    }},
    "temperature": {{
      "type": "number"
    }},
    "humidity": {{
      "type": "number"
    }}
  }},
  "required": [
    "sensor_id",
    "timestamp"
  ]
}}
</schema>

<code>
from datetime import datetime
from typing import TypeAlias
import csv
# TODO: add any other imports here

@dataclass
class MappedData:
    sensor_id: str
    timestamp: datetime 
    pm0_3: float
    pm0_5: float
    pm1: float  
    pm2_5: float
    pm4: float
    pm5: float
    pm10: float
    temperature: float
    humidity: float

MappedDataList: TypeAlias = list(MappedData)

class Converter:
    \"\"\"A class that manages the conversion of a provided set of data, either in json or csv, to the AFRI_SET_COMMON format.
    \"\"\"
    
    def convert(input:str) -> MappedDataList:
        \"\"\"Converts the provided data to the AFRI_SET_COMMON format.

        Parameters
        ----------
        input : str
           Input data to be converted to the AFRI_SET_COMMON format.

        Returns
        -------
        MappedDataList
            A list of line items from the incoming data converted to the AFRI_SET_COMMON format.
        \"\"\"
        # TODO: add implementation here  
</code>

<response>
    <python>
        <!--insert generated python class here -->
    </python>
    <confidence>
        <!--insert confidence rate of transform here -->
    </confidence>
    <test>
        <!--insert executable python code here to test, which must pass -->
    </test>
</response>

[Task instructions]
You ALWAYS follow these guidelines when writing your response:
<guidelines>
- You will be acting as an expert Python software developer writing code compliant with Python 3.10 that reads the input data provided as part of the question and transforms it into the AFRI_SET_COMMON json format as defined by the json schema defined in the <schema></schema> XML tag. 
- The generated Python code should follow the structure as described in the <code></code> XML tags. 
- Use the python csv module to read from the input data.
- The input data will be different with each invocation of the convert function, therefore nothing should be hardcoded within this function. Instead parse the incoming input data to obtain all values.
- Ensure the code is syntactically correct, bug-free, optimized, not span multiple lines unnecessarily, and prefer to use standard libraries. 
- The response must be limited to the structure as described in the <response></response> XML tags. Nothing else (i.e. context, steps, or explanation) should be returned as part of the response.
</guidelines>

When you reply, first determine how the provided input should be mapped to the AFRI_SET_COMMON json format. Write this mapping within the <thinking></thinking> XML tags. This is a space for you to write down relevant content and will not be shown to the user.  Once you are done extracting determing the mapping steps, answer the question.  Put your answer inside the <response></response> XML tags.

Question: {question}

Assistant:"""

MAPPING_PROMPT = PromptTemplate(
    template=mapping_prompt_template, input_variables=["context", "question"]
)

: 

Get the results...

In [ ]:
execute(MAPPING_PROMPT, query_1, '?')

: 

In [ ]:
execute(MAPPING_PROMPT, query_2, '?')

: 

In [ ]:
execute(MAPPING_PROMPT, query_3, '?')

: 

: 